In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install scenedetect[opencv]
!cd /content/drive/MyDrive/Dense-Video-Captioning/

!ls /content/drive/MyDrive/Dense-Video-Captioning/*.py

!cat /content/drive/MyDrive/Dense-Video-Captioning/config.py

import sys
sys.path.append('/content/drive/MyDrive/Dense-Video-Captioning/')
import os
os.chdir('/content/drive/MyDrive')
import config
from scenedetect import VideoManager, SceneManager, StatsManager
from scenedetect.detectors import ContentDetector
from scenedetect.scene_manager import save_images, write_scene_list_html


     |████████████████████████████████| 98 kB 3.6 MB/s 
/content/drive/MyDrive/Dense-Video-Captioning/config.py
/content/drive/MyDrive/Dense-Video-Captioning/extract_features.py
train_path = "/content/drive/MyDrive/Dense-Video-Captioning/data/training_data"
test_path = "/content/drive/MyDrive/Dense-Video-Captioning/data/testing_data/"
batch_size = 320
learning_rate = 0.0007
epochs = 150
latent_dim = 128
num_encoder_tokens = 2560
num_decoder_tokens = 1500
time_steps_encoder = 20
max_probability = -1
save_model_path = 'model_final_2'
validation_split = 0.15
max_length = 10
search_type = 'greedy'


In [3]:
import config
import tensorflow as tf
import shutil
import tqdm
import numpy as np
import cv2
import os
#from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import config
from scenedetect import VideoManager, SceneManager, StatsManager
from scenedetect.detectors import ContentDetector
from scenedetect.scene_manager import save_images, write_scene_list_html



In [ ]:
def video_to_frames(video,startT, stopT):
    startT= startT*1000
    stopT= stopT*1000
    path = os.path.join(config.test_path, 'temporary_images')
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    video_path = os.path.join(config.test_path, 'video', video)
    count = 0
    image_list = []
    # Path to video file
    cap = cv2.VideoCapture(video_path)
    success = True
    while success and cap.get(cv2.CAP_PROP_POS_MSEC) < startT:
      success, image = cap.read()
    while success and cap.get(cv2.CAP_PROP_POS_MSEC) <= stopT:
        ret, frame = cap.read()
        if ret is False:
            break
        cv2.imwrite(os.path.join(config.test_path, 'temporary_images', 'frame%d.jpg' % count), frame)
        image_list.append(os.path.join(config.test_path, 'temporary_images', 'frame%d.jpg' % count))
        count += 1

    cap.release()
    print(image_list)
    #cv2.destroyAllWindows()
    return image_list


In [4]:

def model_cnn_load():
    model= tf.keras.applications.EfficientNetB7(input_shape=(600, 600, 3), include_top=True, weights="imagenet")
    #model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    model.summary()
    return model_final
m= model_cnn_load()
m.summary()

from typing import Type
def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (600, 600))
    return img



268337152/268326632 [==============================] - 2s 0us/step
Model: "efficientnetb7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 600, 600, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 600, 600, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 600, 600, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (Ze

In [ ]:

def extract_features(video, model,startT,stopT):
    """

    :param video: The video whose frames are to be extracted to convert into a numpy array
    :param model: the pretrained vgg16 model
    :return: numpy array of size 4096x80
    """
    video_id = video.split(".")[0]
    print(video_id)
    print(f'extracting frames from {video} in {startT} and {stopT} secs')
    image_list = video_to_frames(video,startT,stopT)
    samples = np.round(np.linspace(0, len(image_list) - 1, 20))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 600, 600, 3))
    for i in range(len(image_list)):
        img = load_image(image_list[i])
        images[i] = img
    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats)
    # cleanup
    shutil.rmtree(os.path.join(config.test_path, 'temporary_images'))
    return img_feats


In [ ]:

def proposal_module(video):
  video_id = video.split(".")[0]
  #print(video_id)
  print(f'Processing video for detecting scenes {video}')
  video_path = os.path.join(config.test_path, 'video', video)
  stats_path = 'result.csv'

  video_manager = VideoManager([video_path])
  stats_manager = StatsManager()
  scene_manager = SceneManager(stats_manager)

  scene_manager.add_detector(ContentDetector(threshold=30))

  video_manager.set_downscale_factor()

  video_manager.start()
  scene_manager.detect_scenes(frame_source=video_manager)

  # result
  with open(stats_path, 'w') as f:
      stats_manager.save_to_csv(f, video_manager.get_base_timecode())

  scene_list = scene_manager.get_scene_list()
  print(f'{len(scene_list)} scenes detected!')

  save_images(
    scene_list,
    video_manager,
    num_images=1,
    image_name_template='$SCENE_NUMBER',
    output_dir='scenes')

  write_scene_list_html('result.html', scene_list)
  scenes=[]
  for scene in scene_list:
      start, end = scene
      scenes.append(float(start.get_seconds()))
      scenes.append(float(end.get_seconds()))
      # your code

  return scenes
#proposal_module('/content/drive/MyDrive/Video-Captioning-main/data/testing_data/video/ecm9gf2Pgkc_1_24.avi')



TEST DATA

In [ ]:
def extract_feats_pretrained_cnn():
      """
      saves the numpy features from all the videos
      """
      model = model_cnn_load()
      print('Model loaded')

      if not os.path.isdir(os.path.join(config.test_path, 'feat_test')):
          os.mkdir(os.path.join(config.test_path, 'feat_test'))
      
      video_list = os.listdir(os.path.join(config.test_path, 'video'))
      feat_label = {}
      feat_label_names= {}
      for video in video_list:
          if not os.path.isdir(os.path.join(config.test_path, 'feat_test')):
            os.makedirs(os.path.join(config.test_path, 'feat_test'))
          
          scenes = proposal_module(video)
          feat_label[str(video)] = len(scenes)/2
          
          count=0
          for i in range(0,len(scenes),2):
            count+=1
            if count==1:
              outfile = os.path.join(config.test_path, 'feat_test', video.split(".")[0] + '.npy')
              feat_label_names[str(video)] = list(str(outfile))
            else:
              outfile = os.path.join(config.test_path, 'feat_test', video.split(".")[0]+"_"+ str(count) + '.npy')
              feat_label_names[str(video)].append(list(str(outfile))) 
            
            
            img_feats = extract_features(video, model,scenes[i],scenes[i+1])
            
            np.save(outfile, img_feats)



In [ ]:
if __name__ == "__main__":
    extract_feats_pretrained_cnn()

import os

for path, subdirs, files in os.walk('/content/drive/MyDrive/Dense-Captioning-main/data/testing_data/feat_test'):
   for filename in files:
     f = os.path.join(path, filename)
     a = open("/content/drive/MyDrive/Dense-Captioning-main/data/testing_data/featId.txt", "a")
     a.write(str(filename)+ '\n')

Output hidden; open in https://colab.research.google.com to view.

#for training set use this code

In [5]:
def video_to_frames(video):
  
    path = os.path.join(config.train_path, 'temporary_images')
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    video_path = os.path.join(config.train_path, '8', video)
    count = 0
    image_list = []
    # Path to video file
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret is False:
            break
        cv2.imwrite(os.path.join(config.train_path, 'temporary_images', 'frame%d.jpg' % count), frame)
        image_list.append(os.path.join(config.train_path, 'temporary_images', 'frame%d.jpg' % count))
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    return image_list


def model_cnn_load():
    model= tf.keras.applications.EfficientNetB7(input_shape=(600, 600, 3), include_top=True, weights="imagenet")
    #model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    model_final.summary()

    return model_final


def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (600, 600))
    return img


def extract_features(video, model):
    """
    :param video: The video whose frames are to be extracted to convert into a numpy array
    :param model: the pretrained vgg16 model
    :return: numpy array of size 4096x80
    """
    video_id = video.split(".")[0]
    print(video_id)
    print(f'Processing video {video}')

    image_list = video_to_frames(video)
    samples = np.round(np.linspace(
        0, len(image_list) - 1, 20))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 600, 600, 3))
    for i in range(len(image_list)):
        img = load_image(image_list[i])
        images[i] = img
    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats)
    # cleanup
    shutil.rmtree(os.path.join(config.train_path, 'temporary_images'))
    return img_feats


def extract_feats_pretrained_cnn():
    """
    saves the numpy features from all the videos
    """
    count=0
    model = model_cnn_load()
    print('Model loaded')

    if not os.path.isdir(os.path.join(config.train_path, 'feat_8')):
        os.mkdir(os.path.join(config.train_path, 'feat_8'))

    video_list = os.listdir(os.path.join(config.train_path, '8'))
    for video in video_list:
        count=count+1
        print(count)
        outfile = os.path.join(config.train_path, 'feat_8', video.split(".")[0] + '.npy')
        img_feats = extract_features(video, model)
        np.save(outfile, img_feats)


if __name__ == "__main__":
    extract_feats_pretrained_cnn()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 600, 600, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 600, 600, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 600, 600, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                            